# This notebook walks through the basics of using TASM.

It uses a (relatively uninteresting) video of a bird with pre-computed object detections.

## Setup: loading a video and detections.

In [ ]:
VIDEO_PATH = 'data/birds.mp4'
DETECTIONS_PATH = 'data/birds.pkl'

### Load detections.

In [ ]:
import pandas as pd

detections = pd.read_pickle(DETECTIONS_PATH)
detections = detections.astype({'x1': int, 'y1': int, 'x2': int, 'y2': int})
detections.head()

### View detections on a few frames.

In [ ]:
import cv2
import matplotlib.pyplot as plt

def draw_box(img, label, x1, y1, x2, y2):
    pt1 = (x1, y1)
    pt2 = (x2, y2)
    cv2.rectangle(img, pt1, pt2, (255, 0, 0), 2)
    cv2.putText(img, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 2, [255, 0, 0], 2)
    return img

num_frames = 5
vc = cv2.VideoCapture(VIDEO_PATH)

for f in range(num_frames):
    ret, frame = vc.read()
    if not ret:
        break
        
    objects = detections[detections.frame == f]
    for i, row in objects.iterrows():
        frame = draw_box(frame, row.label, row.x1, row.y1, row.x2, row.y2)
    
    plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(frame); plt.show()

## Loading the video and detections into TASM.

## Retrieving objects of interest using TASM.